<a href="https://colab.research.google.com/github/AashuYDV/Ireland-Job-Distribution-Analyis/blob/main/Ireland_Job_Distribution_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas python-dateutil


In [ ]:
import requests
import pandas as pd
import time
import hashlib
from datetime import datetime, timedelta
from dateutil import parser


In [ ]:
APP_ID = "026f28a2"
APP_KEY = "ea3d0a8f3077b07358a4169e021c3e36"

BASE_URL = "https://api.adzuna.com/v1/api/jobs/gb/search"
PAGES_PER_QUERY = 2        # keep small for Phase-1
RESULTS_PER_PAGE = 50
SLEEP_SECONDS = 1.2

PIPELINE_VERSION = "v1.0-uk-phase1"
RUN_ID = datetime.utcnow().strftime("%Y%m%d_uk_run_01")
INGESTION_DATE = datetime.utcnow().date().isoformat()


/tmp/ipython-input-328557299.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = datetime.utcnow().strftime("%Y%m%d_uk_run_01")
/tmp/ipython-input-328557299.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  INGESTION_DATE = datetime.utcnow().date().isoformat()


In [ ]:
ROLE_FAMILIES = {
    "Engineering": [
        "software engineer",
        "devops engineer",
        "mechanical engineer",
        "electrical engineer"
    ],
    "Data": [
        "data analyst",
        "data scientist",
        "analytics"
    ],
    "Product": [
        "product manager",
        "product owner"
    ],
    "Business": [
        "business analyst",
        "consultant",
        "operations manager"
    ],
    "Marketing": [
        "marketing manager",
        "digital marketing"
    ],
    "Finance": [
        "accountant",
        "finance analyst"
    ],
    "Healthcare": [
        "nurse",
        "clinical research",
        "pharma"
    ],
    "Design": [
        "designer",
        "ux designer"
    ],
    "HR": [
        "human resources",
        "hr manager"
    ]
}


In [ ]:
def generate_record_id(source, title, company, location):
    raw = f"{source}|{title}|{company}|{location}"
    return hashlib.md5(raw.encode()).hexdigest()


def normalize_title(title):
    return title.lower().strip()


def extract_experience_level(title):
    t = title.lower()
    if "senior" in t:
        return "senior"
    if "junior" in t or "graduate" in t:
        return "entry"
    if "lead" in t or "manager" in t:
        return "senior"
    return "mid"


In [ ]:
all_rows = []

for role_family, queries in ROLE_FAMILIES.items():
    print(f"\n🔍 ROLE FAMILY: {role_family}")

    for query in queries:
        print(f"   → Query: {query}")

        for page in range(1, PAGES_PER_QUERY + 1):
            params = {
                "app_id": APP_ID,
                "app_key": APP_KEY,
                "results_per_page": RESULTS_PER_PAGE,
                "what": query,
                "content-type": "application/json"
            }

            url = f"{BASE_URL}/{page}"
            response = requests.get(url, params=params)

            print(f"      Page {page} | Status {response.status_code}")

            if response.status_code != 200:
                print("      ❌ Skipped")
                break

            data = response.json()
            results = data.get("results", [])

            if not results:
                break

            for job in results:
                title = job.get("title", "")
                company = job.get("company", {}).get("display_name", "")
                location = job.get("location", {}).get("display_name", "")
                city = location.split(",")[0] if location else None
                description = job.get("description", "")
                url = job.get("redirect_url")
                created = job.get("created")

                try:
                    posted_date = parser.parse(created).date().isoformat() if created else None
                except:
                    posted_date = None

                record_id = generate_record_id("adzuna", title, company, location)

                all_rows.append({
                    # Identity
                    "record_id": record_id,
                    "source": "adzuna",
                    "source_platform": job.get("source", {}).get("name", "unknown"),
                    "job_url": url,

                    # Job content
                    "job_title_raw": title,
                    "job_title_normalized": normalize_title(title),
                    "role_category": role_family,
                    "job_description": description,
                    "skills": None,
                    "employment_type": None,
                    "work_mode": None,
                    "experience_level": extract_experience_level(title),

                    # Location
                    "country": "UK",
                    "location_raw": location,
                    "location_city": city,
                    "location_region": None,

                    # Time
                    "posted_date": posted_date,
                    "first_seen_date": INGESTION_DATE,
                    "last_seen_date": INGESTION_DATE,
                    "is_active": True,
                    "days_listed": None,

                    # Market
                    "market_count": data.get("count", None),
                    "confidence_score": 0.65,

                    # Metadata
                    "ingestion_date": INGESTION_DATE,
                    "run_id": RUN_ID,
                    "pipeline_version": PIPELINE_VERSION,
                    "query_used": query,
                    "country_code_used": "gb",
                    "page_fetched": page,
                    "api_status": response.status_code,
                    "data_quality_flag": "raw",
                    "validation_notes": None
                })

            time.sleep(SLEEP_SECONDS)



🔍 ROLE FAMILY: Engineering
   → Query: software engineer
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: devops engineer
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: mechanical engineer
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: electrical engineer
      Page 1 | Status 200
      Page 2 | Status 200

🔍 ROLE FAMILY: Data
   → Query: data analyst
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: data scientist
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: analytics
      Page 1 | Status 200
      Page 2 | Status 200

🔍 ROLE FAMILY: Product
   → Query: product manager
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: product owner
      Page 1 | Status 200
      Page 2 | Status 200

🔍 ROLE FAMILY: Business
   → Query: business analyst
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: consultant
      Page 1 | Status 200
      Page 2 | Status 200
   → Query: operations manag

In [6]:
df = pd.DataFrame(all_rows)

if df.empty:
    raise RuntimeError("❌ No UK jobs fetched. Check API credentials or quota.")

df = df.drop_duplicates(subset=["record_id"])

output_file = "uk_job_signal_raw.csv"
df.to_csv(output_file, index=False)

print(f"\n✅ Saved {len(df)} rows to {output_file}")
df.head()


NameError: name 'all_rows' is not defined

In [7]:
# ================================
# CELL 7: AGGREGATE OPEN ROLES
# ================================

# Safety check
required_cols = [
    "record_id",
    "role_category",
    "job_title_normalized",
    "location_city",
    "market_count"
]

missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise RuntimeError(f"❌ Missing required columns: {missing}")

# Aggregate
df_agg = (
    df.groupby(["role_category", "job_title_normalized"])
      .agg(
          open_role_postings=("record_id", "nunique"),
          market_count_estimate=("market_count", "median"),
          cities_covered=("location_city", lambda x: x.nunique())
      )
      .reset_index()
      .sort_values("open_role_postings", ascending=False)
)

# Save aggregated CSV
output_agg_file = "uk_open_roles_aggregated.csv"
df_agg.to_csv(output_agg_file, index=False)

print(f"✅ Aggregated file saved as {output_agg_file}")
df_agg.head(10)


✅ Aggregated file saved as uk_open_roles_aggregated.csv


,role_category,job_title_normalized,open_role_postings,market_count_estimate,cities_covered
270,Engineering,devops engineer,73,1453.0,41
63,Business,operations manager,67,37297.0,52
742,Product,product owner,67,2180.0,44
136,Data,data analyst,63,2704.0,46
258,Design,ux designer,52,1522.0,32
9,Business,business analyst,48,3721.0,35
723,Product,product manager,47,15682.0,29
656,Marketing,digital marketing executive,43,7144.0,39
350,Finance,finance analyst,37,1501.0,28
155,Data,data scientist,36,5142.0,12


In [8]:
import pandas as pd

df = pd.read_csv("uk_job_signal_raw.csv")


In [9]:
import re

# ================================
# WORK MODE INFERENCE
# ================================

def infer_work_mode(text):
    t = str(text).lower()
    if any(x in t for x in ["remote", "work from home", "wfh", "fully remote"]):
        return "remote"
    if any(x in t for x in ["hybrid", "flexible working", "mixed"]):
        return "hybrid"
    if any(x in t for x in ["on-site", "onsite", "office-based", "in office"]):
        return "onsite"
    return "unknown"


# ================================
# EMPLOYMENT TYPE INFERENCE
# ================================

def infer_employment_type(text):
    t = str(text).lower()
    if any(x in t for x in ["full-time", "full time", "permanent"]):
        return "full-time"
    if any(x in t for x in ["part-time", "part time"]):
        return "part-time"
    if any(x in t for x in ["contract", "contractor", "fixed-term"]):
        return "contract"
    if any(x in t for x in ["intern", "internship", "graduate"]):
        return "internship"
    return "unknown"


# ================================
# SKILL EXTRACTION
# ================================

GENERAL_SKILLS = [
    "python","sql","excel","aws","azure","gcp","java","javascript",
    "react","node","docker","kubernetes","git","linux",
    "power bi","tableau","machine learning","data analysis",
    "communication","stakeholder management","problem solving",
    "agile","scrum","jira"
]

ROLE_DEFAULT_SKILLS = {
    "Engineering": ["software development", "system design", "debugging"],
    "Data": ["data analysis", "sql", "reporting"],
    "Product": ["roadmapping", "stakeholder management", "agile"],
    "Business": ["process analysis", "requirements gathering"],
    "Marketing": ["campaign management", "seo", "analytics"],
    "Finance": ["financial analysis", "accounting"],
    "Healthcare": ["patient care", "clinical compliance"],
    "Design": ["ui design", "ux research"],
    "HR": ["recruitment", "employee relations"]
}

def extract_skills(text, role_category):
    t = str(text).lower()
    found = [skill for skill in GENERAL_SKILLS if skill in t]

    if found:
        return ", ".join(sorted(set(found)))

    # fallback to role defaults
    defaults = ROLE_DEFAULT_SKILLS.get(role_category, [])
    return ", ".join(defaults) if defaults else "unknown"


# ================================
# SALARY EXTRACTION (RAW TEXT)
# ================================

SALARY_REGEX = re.compile(
    r"(£\s?\d{2,3}[kK]|\£\s?\d{2,3},\d{3}|\£\s?\d{2,3}\s?-\s?\£?\d{2,3}[kK]|\£\s?\d+\/day)",
    re.IGNORECASE
)

def extract_salary(text):
    matches = SALARY_REGEX.findall(str(text))
    return matches[0] if matches else "unknown"


# ================================
# APPLY ENRICHMENT
# ================================

df["work_mode"] = df["job_description"].apply(infer_work_mode)
df["employment_type"] = df["job_description"].apply(infer_employment_type)
df["skills"] = df.apply(
    lambda x: extract_skills(x["job_description"], x["role_category"]),
    axis=1
)
df["salary_raw"] = df["job_description"].apply(extract_salary)

print("✅ Enrichment completed")


✅ Enrichment completed


In [10]:
df.to_csv("uk_job_signal_enriched.csv", index=False)
print("📁 Saved: uk_job_signal_enriched.csv")


📁 Saved: uk_job_signal_enriched.csv


In [11]:
import pandas as pd
import json
import re

# ============================
# 1. INPUT: RAW DATAFRAME
# ============================
# df_raw must already exist
# Example:
df_raw = pd.read_csv("uk_job_signal_raw.csv")

df = df_raw.copy()

# Safety cleanup
df = df.dropna(subset=["job_title_normalized"])
df["job_title"] = df["job_title_normalized"].str.strip().str.title()

# ============================
# 2. SKILL & SIGNAL DICTIONARIES
# ============================
SKILL_KEYWORDS = {
    "python": "Python",
    "sql": "SQL",
    "excel": "Excel",
    "aws": "AWS",
    "azure": "Azure",
    "gcp": "GCP",
    "react": "React",
    "node": "Node.js",
    "java": "Java",
    "c++": "C++",
    "power bi": "Power BI",
    "tableau": "Tableau",
    "ml": "Machine Learning",
    "data analysis": "Data Analysis"
}

DEFAULT_SKILLS_BY_ROLE = {
    "Engineering": ["Problem Solving", "System Design"],
    "Data": ["SQL", "Data Analysis"],
    "Product": ["Roadmapping", "Stakeholder Management"],
    "Business": ["Analysis", "Communication"],
    "Marketing": ["SEO", "Campaigns"],
    "Finance": ["Accounting", "Reporting"],
    "Healthcare": ["Clinical Knowledge"],
    "Design": ["UX", "UI"],
    "HR": ["Recruitment", "People Management"]
}

def extract_skills(text, role_category):
    text = str(text).lower()
    skills = set()

    for k, v in SKILL_KEYWORDS.items():
        if re.search(rf"\b{k}\b", text):
            skills.add(v)

    if not skills:
        skills.update(DEFAULT_SKILLS_BY_ROLE.get(role_category, []))

    return sorted(skills)

def infer_work_mode(text):
    text = str(text).lower()
    if "hybrid" in text:
        return "Hybrid"
    if "remote" in text:
        return "Remote"
    return "On-site"

def infer_experience(text):
    text = str(text).lower()
    if "graduate" in text or "entry" in text:
        return "Entry"
    if "senior" in text or "lead" in text:
        return "Senior"
    return "Mid"

# ============================
# 3. ENRICH RAW DATA
# ============================
df["skills"] = df.apply(
    lambda x: extract_skills(x.get("job_description", ""), x.get("role_category", "")),
    axis=1
)

df["work_mode"] = df["job_description"].apply(infer_work_mode)
df["experience_level"] = df["job_description"].apply(infer_experience)

# ============================
# 4. AGGREGATION LOGIC
# ============================
agg = (
    df.groupby(["job_title", "role_category"])
    .agg(
        open_roles=("job_url", "count"),
        top_locations=("location_city", lambda x: ", ".join(x.dropna().unique()[:3])),
        skills=("skills", lambda x: sorted(set(sum(x, [])))),
        work_mode=("work_mode", lambda x: x.value_counts().idxmax()),
        experience_level=("experience_level", lambda x: x.value_counts().idxmax()),
        job_links=("job_url", lambda x: json.dumps(
            [{"source": "Adzuna", "url": u} for u in x.dropna().unique()]
        ))
    )
    .reset_index()
)

# ============================
# 5. FINAL COLUMN ORDER
# ============================
agg = agg[
    [
        "job_title",
        "role_category",
        "open_roles",
        "top_locations",
        "experience_level",
        "work_mode",
        "skills",
        "job_links"
    ]
]

# ============================
# 6. EXPORT
# ============================
OUTPUT_FILE = "uk_open_roles_aggregated.csv"
agg.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Aggregated CSV created: {OUTPUT_FILE}")
agg.head()


✅ Aggregated CSV created: uk_open_roles_aggregated.csv


,job_title,role_category,open_roles,top_locations,experience_level,work_mode,skills,job_links
0,60Th Anniversary Chair (Professor) In Human Re...,HR,1,Manchester Science Park,Senior,On-site,"[People Management, Recruitment]","[{""source"": ""Adzuna"", ""url"": ""https://www.adzu..."
1,A&E Consultant,Business,1,Milton Keynes,Senior,On-site,"[Analysis, Communication]","[{""source"": ""Adzuna"", ""url"": ""https://www.adzu..."
2,Account Development Manager,Healthcare,1,Reading,Senior,On-site,[Clinical Knowledge],"[{""source"": ""Adzuna"", ""url"": ""https://www.adzu..."
3,Accountant,Finance,17,"Haverfordwest, Hereford, Edinburgh",Mid,On-site,"[Accounting, Reporting]","[{""source"": ""Adzuna"", ""url"": ""https://www.adzu..."
4,Accountant - Accounts Preparation,Finance,2,Belfast,Mid,On-site,"[Accounting, Reporting]","[{""source"": ""Adzuna"", ""url"": ""https://www.adzu..."
